<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/c_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
#Librerias

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import joblib
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from matplotlib import pyplot
from sklearn.model_selection import RandomizedSearchCV

In [87]:
file_path = '/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/df_mergedtriple.csv'

# Leer el archivo CSV en un DataFrame
df_mergedtriple = pd.read_csv(file_path)

# Mostrar las primeras filas del DataFrame para verificar la carga correcta
print(df_mergedtriple.head())


   Age     BusinessTravel  DistanceFromHome  Education EducationField  \
0   51      Travel_Rarely                 6          2  Life Sciences   
1   51      Travel_Rarely                 6          2  Life Sciences   
2   31  Travel_Frequently                10          1  Life Sciences   
3   32  Travel_Frequently                17          4          Other   
4   32  Travel_Frequently                17          4          Other   

   EmployeeID  JobLevel MaritalStatus  MonthlyIncome  NumCompaniesWorked  ...  \
0           1         1       Married         131160                 1.0  ...   
1           1         1       Married         131160                 1.0  ...   
2           2         1        Single          41890                 0.0  ...   
3           3         4       Married         193280                 1.0  ...   
4           3         4       Married         193280                 1.0  ...   

   TrainingTimesLastYear  YearsAtCompany  YearsSinceLastPromotion  \
0    

In [88]:
df_mergedtriple.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   DistanceFromHome         8820 non-null   int64  
 3   Education                8820 non-null   int64  
 4   EducationField           8820 non-null   object 
 5   EmployeeID               8820 non-null   int64  
 6   JobLevel                 8820 non-null   int64  
 7   MaritalStatus            8820 non-null   object 
 8   MonthlyIncome            8820 non-null   int64  
 9   NumCompaniesWorked       8820 non-null   float64
 10  PercentSalaryHike        8820 non-null   int64  
 11  StockOptionLevel         8820 non-null   int64  
 12  TotalWorkingYears        8820 non-null   float64
 13  TrainingTimesLastYear    8820 non-null   int64  
 14  YearsAtCompany          

In [93]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Lista de variables categóricas (tanto nominales como ordinales que se convertirán en dummies)
list_cat_nominales = ['BusinessTravel', 'EducationField', 'MaritalStatus']
list_cat_ordinales = ['Education', 'JobLevel', 'StockOptionLevel']

# Variables numéricas que se estandarizarán
list_numeric = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
                'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
                'YearsSinceLastPromotion', 'YearsWithCurrManager',
                'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'JobInvolvement']

# Convertir variables categóricas a dummies
df_dummies = pd.get_dummies(df_mergedtriple[list_cat_nominales + list_cat_ordinales], drop_first=True)

# Convertir cualquier valor booleano en df_dummies a 0 y 1
df_dummies = df_dummies.astype(int)

# Estandarizar las variables numéricas
scaler = StandardScaler()
df_numeric_scaled = pd.DataFrame(scaler.fit_transform(df_mergedtriple[list_numeric]), columns=list_numeric)

# Añadir la variable Year al DataFrame final
df_final = pd.concat([df_numeric_scaled, df_dummies, df_mergedtriple[['EmployeeID', 'Attrition', 'Year']]], axis=1)

# Mostrar el DataFrame final listo para el modelo
df_final.head()


,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,...,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single,EmployeeID,Attrition,Year
0,1.541369,-0.393938,1.405136,-0.678464,-1.150554,-1.322079,2.483396,-0.981014,-0.679146,-1.155935,...,1,0,0,0,0,1,0,1,0,2015
1,1.541369,-0.393938,1.405136,-0.678464,-1.150554,-1.322079,2.483396,-0.981014,-0.679146,-1.155935,...,1,0,0,0,0,1,0,1,0,2016
2,-0.648668,0.099639,-0.491661,-1.079486,2.129306,-0.678877,0.155707,-0.327893,-0.368715,-0.034520,...,1,0,0,0,0,0,1,2,1,2015
3,-0.539166,0.963398,2.725053,-0.678464,-0.057267,-0.807517,-0.620189,-0.327893,-0.679146,-0.314873,...,0,0,0,1,0,1,0,3,0,2015
4,-0.539166,0.963398,2.725053,-0.678464,-0.057267,-0.807517,-0.620189,-0.327893,-0.679146,-0.314873,...,0,0,0,1,0,1,0,3,0,2016


In [91]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Age                               8820 non-null   float64
 1   DistanceFromHome                  8820 non-null   float64
 2   MonthlyIncome                     8820 non-null   float64
 3   NumCompaniesWorked                8820 non-null   float64
 4   PercentSalaryHike                 8820 non-null   float64
 5   TotalWorkingYears                 8820 non-null   float64
 6   TrainingTimesLastYear             8820 non-null   float64
 7   YearsAtCompany                    8820 non-null   float64
 8   YearsSinceLastPromotion           8820 non-null   float64
 9   YearsWithCurrManager              8820 non-null   float64
 10  EnvironmentSatisfaction           8820 non-null   float64
 11  JobSatisfaction                   8820 non-null   float64
 12  WorkLi

In [94]:
from sklearn.model_selection import train_test_split

# Preparar datos para entrenamiento y prueba
# Filtrar datos para los años de entrenamiento y validación (2015 y 2016)
df_train_val = df_final[df_final['Year'] < 2016]
df_test = df_final[df_final['Year'] == 2016]

# Dividir los datos de entrenamiento y validación
X = df_train_val.drop(columns=['Attrition', 'Year', 'EmployeeID'])
y = df_train_val['Attrition']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Filtrar los datos de 2016 para prueba (predicción)
X_test = df_test.drop(columns=['Attrition', 'Year', 'EmployeeID'])  # Excluir EmployeeID aquí
employee_ids_test = df_test['EmployeeID']  # Guardar EmployeeID separado

# Guardar los archivos CSV
X_train.to_csv(file_path + 'X_train.csv', index=False)
y_train.to_csv(file_path + 'y_train.csv', index=False)
X_val.to_csv(file_path + 'X_val.csv', index=False)
y_val.to_csv(file_path + 'y_val.csv', index=False)
X_test.to_csv(file_path + 'X_test.csv', index=False)
employee_ids_test.to_csv(file_path + 'employee_ids_test.csv', index=False)  # Guardar EmployeeID


In [95]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Definir la ruta de carga
file_path = '/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/'

# Cargar los datos
X_train = pd.read_csv(file_path + 'X_train.csv')
y_train = pd.read_csv(file_path + 'y_train.csv').squeeze()  # Convertir de DataFrame a Serie
X_val = pd.read_csv(file_path + 'X_val.csv')
y_val = pd.read_csv(file_path + 'y_val.csv').squeeze()  # Convertir de DataFrame a Serie

# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Crear y entrenar el modelo de Regresión Logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_logistic = logistic_model.predict(X_val_scaled)

# Evaluar el modelo
print("Matriz de Confusión:")
print(confusion_matrix(y_val, y_pred_logistic))

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_logistic))

# AUC-ROC
y_prob_logistic = logistic_model.predict_proba(X_val_scaled)[:, 1]
print("\nAUC-ROC Score:", roc_auc_score(y_val, y_prob_logistic))


Matriz de Confusión:
[[746  11]
 [105  20]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       757
           1       0.65      0.16      0.26       125

    accuracy                           0.87       882
   macro avg       0.76      0.57      0.59       882
weighted avg       0.84      0.87      0.83       882


AUC-ROC Score: 0.7369722589167768


In [96]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definir la ruta de carga
file_path = '/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/'

# Cargar los datos
X_train = pd.read_csv(file_path + 'X_train.csv')
y_train = pd.read_csv(file_path + 'y_train.csv').squeeze()  # Convertir de DataFrame a Serie
X_val = pd.read_csv(file_path + 'X_val.csv')
y_val = pd.read_csv(file_path + 'y_val.csv').squeeze()  # Convertir de DataFrame a Serie

# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Crear y entrenar el modelo de Árbol de Decisión
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_tree = decision_tree_model.predict(X_val_scaled)

# Evaluar el modelo
print("Matriz de Confusión:")
print(confusion_matrix(y_val, y_pred_tree))

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_pred_tree))

# AUC-ROC
y_prob_tree = decision_tree_model.predict_proba(X_val_scaled)[:, 1]
print("\nAUC-ROC Score:", roc_auc_score(y_val, y_prob_tree))


Matriz de Confusión:
[[746  11]
 [ 28  97]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       757
           1       0.90      0.78      0.83       125

    accuracy                           0.96       882
   macro avg       0.93      0.88      0.90       882
weighted avg       0.95      0.96      0.95       882


AUC-ROC Score: 0.9634557463672391


In [97]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import resample

# Generar un conjunto de datos sintético
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, n_clusters_per_class=1, weights=[0.8, 0.2], flip_y=0, random_state=42)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Mostrar la distribución de clases en y_train y y_test antes del balanceo
print("Distribución de clases en y_train antes del balanceo:")
print(pd.Series(y_train).value_counts())
print("Distribución de clases en y_test antes del balanceo:")
print(pd.Series(y_test).value_counts())

# Balanceo de clases en el conjunto de entrenamiento
# Combinar X_train y y_train en un DataFrame para facilitar el balanceo
train_df = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])
train_df['target'] = y_train

# Separar clases mayoritaria y minoritaria
majority_class = train_df[train_df['target'] == 0]
minority_class = train_df[train_df['target'] == 1]

# Realizar sobremuestreo de la clase minoritaria
minority_class_upsampled = resample(minority_class,
                                    replace=True,  # Muestreo con reemplazo
                                    n_samples=len(majority_class),  # Igualar el tamaño de la clase mayoritaria
                                    random_state=42)  # Para reproducibilidad

# Combinar clase mayoritaria y la clase minoritaria sobremuestreada
upsampled_train_df = pd.concat([majority_class, minority_class_upsampled])

# Separar las características y la variable objetivo del DataFrame balanceado
X_train_balanced = upsampled_train_df.drop('target', axis=1).values
y_train_balanced = upsampled_train_df['target'].values

# Verificar el balanceo de clases en el conjunto de entrenamiento después del balanceo
print("Distribución de clases en y_train después del balanceo:")
print(pd.Series(y_train_balanced).value_counts())

# Configurar GridSearchCV para afinación de hiperparámetros
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Definir el modelo base
decision_tree = DecisionTreeClassifier()

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=decision_tree,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

# Ajustar GridSearchCV
grid_search.fit(X_train_balanced, y_train_balanced)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Usar el mejor modelo encontrado
best_tree_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_tree_model.predict(X_test)

# Evaluar el modelo
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Evaluar el modelo con AUC-ROC
if len(np.unique(y_test)) == 2:  # Asegurarse de que hay ambas clases en y_test
    print("\nAUC-ROC Score:", roc_auc_score(y_test, best_tree_model.predict_proba(X_test)[:, 1]))
else:
    print("\nAUC-ROC Score no definido debido a la falta de clases en y_test.")


Distribución de clases en y_train antes del balanceo:
0    5592
1    1408
Name: count, dtype: int64
Distribución de clases en y_test antes del balanceo:
0    2408
1     592
Name: count, dtype: int64
Distribución de clases en y_train después del balanceo:
0    5592
1    5592
Name: count, dtype: int64
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros encontrados:
{'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 4, 'min_samples_split': 10}
Matriz de Confusión:
[[2398   10]
 [  21  571]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2408
           1       0.98      0.96      0.97       592

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000


AUC-ROC Score: 0.9829748249079645


In [110]:
y_test_pred = best_tree_model.predict(X_test)
pd.DataFrame({'EmployeeID': employee_ids_test, 'PredictedAttrition': y_test_pred}).to_csv(file_path + 'predictions_test.csv', index=False)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 26 features, but DecisionTreeClassifier is expecting 20 features as input.

In [112]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (7000, 20)
X_test shape: (4410, 26)
